In [ ]:
!pip install ultralytics wandb roboflow -q

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="Nj0TPvJWofa4sHFzyt5L")
project = rf.workspace("kelvin-snkig").project("person-helmet-and-vest")
version = project.version(4)
dataset = version.download("yolov11")

In [ ]:
from ultralytics import settings

settings.update({"wandb": True})

print(settings['wandb'])

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [ ]:
import wandb

wandb.login(key=user_secrets.get_secret("wandb"))

In [ ]:
project = "PPE Detection Model"

In [ ]:
api = wandb.Api()

runs = api.runs(f'kelpinnnnnn/{project}')

run_ids = []

for run in runs[:10]:
    print(run.id, run.name)
    run_ids.append(run.id)

In [ ]:
for id in run_ids:
    try:
        # Coba download dari project pertama
        artifact = api.artifact(f'kelpinnnnnn/{project}/run_{id}_model:v0', type='model')
        artifact.download()
        print(f"✅ Success: {id} added to run_ids from {project}")
    except Exception as e1:
        print(f"❌ Failed from {project} for run {id}: {e1}")
        try:
            # Coba ulang dari project kedua jika yang pertama gagal
            print("🔁 Trying Ultralytics project...")
            artifact = api.artifact(f'kelpinnnnnn/Ultralytics/run_{id}_model:v0', type='model')
            artifact.download()
            print(f"✅ Success: {id} added to run_ids from Ultralytics")
        except Exception as e2:
            print(f"❌ Failed from Ultralytics for run {id}: {e2}")

In [ ]:
import os
from ultralytics import YOLO
import json
from pathlib import Path
import cv2

# Daftar 2 gambar untuk dideteksi
imgs_path_to_detect = [
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/IMG_20250305_100034_jpg.rf.5fc6b9bd940f23983e747d89de91fa20.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/IMG_20250319_094958_jpg.rf.56b9b6db03611a8e5777a588991edc6e.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/IMG_20250528_100425_jpg.rf.875da096f7895ba3074e1625f6d473aa.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/IMG_20250528_100449_jpg.rf.1bd395809fc1879335b4704cb1d07b0b.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/IMG_20250528_100526_jpg.rf.f79fa70af4baa16e521181a10f8a7088.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/IMG_20250528_100535_jpg.rf.0ea29889db40b5ca22d59cdefa8c7d37.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/IMG_20250528_100607_jpg.rf.3fde96c45940b62205b907f432e32c34.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/IMG_20250528_100800_1_jpg.rf.b67a4aa92530b48ac51a1e78f980ba9e.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/IMG_20250528_100805_jpg.rf.4aefad71640483128336439d3b984dc1.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/IMG_20250528_101211_jpg.rf.7e8e1aab085746b4b8804661b2419c09.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/IMG_20250528_101211_jpg.rf.7e8e1aab085746b4b8804661b2419c09.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/WhatsApp-Image-2025-06-04-at-12_52_45_9ecc2f9d_jpg.rf.850e071ebd2bcf5eb3644a58dbe69663.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/WhatsApp-Image-2025-06-04-at-12_53_15_a63fcde4_jpg.rf.f729eea89e34c0c80918b09378e6c5fd.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/WhatsApp-Image-2025-06-04-at-12_54_44_d14bf4c1_jpg.rf.602de2b3f11479e698dfa95cabfd7cb9.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/WhatsApp-Image-2025-06-04-at-12_56_28_10055a3c_jpg.rf.7d4f24aa73d96b80a87329bac39be698.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/WhatsApp-Image-2025-06-04-at-12_58_29_93660b05_jpg.rf.ce8531ee4bbd5eb12bb3b2ba9fcceca2.jpg",
    "/kaggle/working/Person,-Helmet,-And-Vest-4/valid/images/WhatsApp-Image-2025-06-04-at-12_59_29_fdab59af_jpg.rf.1fc41fa78d8c90462bfe14ac2289c6cc.jpg"
]

# Direktori output
output_base_dir = "/kaggle/working/output_hasil_deteksi"
os.makedirs(output_base_dir, exist_ok=True)

# Simpan hasil deteksi semua run
all_results = {}

project = "PPE Detection Model"
# Loop semua run
for id in run_ids:
    run = api.run(f"kelpinnnnnn/{project}/{id}")
    run_name = run.name
    model_path = f"/kaggle/working/artifacts/run_{id}_model:v0"
    model = YOLO(f"{model_path}/best.pt")

    run_output_dir = Path(output_base_dir) / run_name
    run_output_dir.mkdir(parents=True, exist_ok=True)

    all_results[run_name] = {}

    for img_path in imgs_path_to_detect:
        results = model.predict(source=img_path, imgsz=640)
        plotted_img = results[0].plot()

        # Simpan gambar hasil deteksi
        img_name = os.path.basename(img_path)
        output_img_path = run_output_dir / img_name
        cv2.imwrite(str(output_img_path), plotted_img)

        # Ambil hasil deteksi dan simpan dalam dict
        detections = []
        for box in results[0].boxes:
            xyxy = box.xyxy[0].tolist()
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            detections.append({
                "bbox": xyxy,
                "confidence": conf,
                "class_id": cls
            })

        all_results[run_name][img_name] = detections

# Simpan semua hasil ke dalam 1 file JSON
output_json_path = f"{output_base_dir}/hasil_deteksi_semua_run.json"
with open(output_json_path, "w") as f:
    json.dump(all_results, f, indent=2)

print(f"✅ Semua hasil disimpan di: {output_base_dir}")
print(f"📄 Hasil JSON: {output_json_path}")

In [ ]:
import json

# Simpan seluruh hasil ke file JSON
output_json_path = "/kaggle/working/output_hasil_deteksi/results_all_models.json"
with open(output_json_path, "w") as f:
    json.dump(all_results, f, indent=2)

print(f"✅ Semua hasil deteksi disimpan ke: {output_json_path}")

In [ ]:
!zip -r "output_hasil_deteksi.zip" "output_hasil_deteksi"